#  Tradução de Textos - Experimento
## Utilização do modelo [MarianMT](https://huggingface.co/transformers/model_doc/marian.html) para tradução. 


* Neste exempo a tradução é feito do inglês para o português, mas ela pode ser feita em qualquer uma das línguas suportadas pelo MarianMT. 
* Para adaptar para traduções em outras línguas é necessário verificacar se há o modelo pré treinado disponível no MarianMT e adaptar o truncamento de strings do [spacy](https://spacy.io/usage/models) para o idioma desejado
* A métrica computada é o [sacrebleu](https://https://github.com/mjpost/sacrebleu) 


### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [1]:
# dataset = "/tmp/data/paracrawl_en_pt_test.csv" #@param {type:"string"}
dataset = "pt_en_testing_texts.csv" 
text = "text_english" #@param {type:"string", label:"Atributo do texto", description:"Este atributo será traduzido e apresentado o resultado."}
target = "text_portuguese" #@param {type:"string", label:"Atributo alvo", description:"Seu modelo será validado com os valores do alvo."}
input_language = "Português" #@param ["Alemão", "Catalão", "Espanhol", "Francês", "Inglês", "Italiano", "Latim", "Português", "Romeno"] {type:"string", label:"Idioma de entrada"}
target_language = "Inglês" #@param ["Alemão", "Catalão", "Espanhol", "Francês", "Inglês", "Italiano", "Latim", "Português", "Romeno"] {type:"string", label:"Idioma de saída"}

#Hyperparams 
seed = 42 #@param {type:"integer",label"Semente de aleatoriedade"}
max_length = 256 #@param {type:"integer",label"Tamanho máximo da sentença de entrada que entrará no MarianMT. Se a sentença for maior ela será quebrada"}
#model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en' #@param ["Helsinki-NLP/opus-mt-ROMANCE-en","Helsinki-NLP/opus-mt-en-ROMANCE"] {type:"integer",label:"Helsinki/NLP model name"}
inference_batch_size = 2 #@param {type:"integer",label"Tamanho do Batch de inferência"}

In [2]:
if input_language == target_language:
    raise Exception('Idioma de entrada e de saída não podem ser iguais')

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [3]:
import pandas as pd

df = pd.read_csv(dataset)

## Formatar dados

Dados nulos serão removidos e serão separados em colunas de teste e validação.

In [4]:
df = df.dropna()
X = df[text].to_numpy()

y = None
if target in df.columns:
    y = df[target].to_numpy()

## Verificando as configurações do MarianMT

- Verificando disponibilidade de GPU e status de hardware
- Instanciando modelo e tokenizador
- Opções de tradução de idiomas
- Modelos pré treinados disponíveis

In [5]:
from multiprocessing import cpu_count
import torch

dev = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(dev)

if dev == "cpu":
    print(f"number of CPU cores: {cpu_count()}")
else:
    print(f"GPU: {torch.cuda.get_device_name(0)}, number of CPU cores: {cpu_count()}")

GPU: Tesla T4, number of CPU cores: 8


## Chamada da Classe MarianMT

In [6]:
if input_language=='Inglês' and target_language!='Inglês':
    models_list = ["Helsinki-NLP/opus-mt-en-ROMANCE"]
elif input_language!='Inglês' and target_language=='Inglês':
    models_list = ["Helsinki-NLP/opus-mt-ROMANCE-en"]
else:
    models_list = ["Helsinki-NLP/opus-mt-ROMANCE-en","Helsinki-NLP/opus-mt-en-ROMANCE"]    

In [7]:
from marianmt_model import MarianMTTranslator

    
hyperparams = {'max_length': max_length, 
               'inference_batch_size': inference_batch_size,
               'target_language':target_language,
               'models_list':models_list,
                'seed':seed
              }

marianmt_model = MarianMTTranslator(hyperparams)

In [8]:
if y is not None:
    aux = marianmt_model.get_result_dataframe(X,y)
else:
    aux = marianmt_model.predict(X)

In [9]:
aux

,source_text,target_text,translated_text,bleu_score
0,"In this way, the civil life of a nation mature...","Deste modo, a vida civil de uma nação amadurec...","In this way, the civil life of a mature nation...",1.924085
1,"1999 XIII. Winnipeg, Canada July 23 to August 8","1999 XIII. Winnipeg, Canadá 23 de julho a 8 de...","1999 XIII.Winnipeg, Canada July 23 to August 8",31.818770
2,"In the mystery of Christmas, Christ's light sh...","No mistério do Natal, a luz de Cristo irradia-...","In the mystery of Christmas, Christ's light sh...",2.167132
3,making it viable to drill two new boreholes in...,e tem o objetivo de viabilizar a perfuração de...,making it viable to drill two new boreholes in...,2.151593


## Salva métricas

Utiliza a função `save_metrics` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar métricas. Por exemplo: `accuracy`, `precision`, `r2_score`, `custom_score` etc.<br>

In [11]:
from platiagro import save_metrics

if y is not None:
    save_metrics(avg_bleu=marianmt_model.avg_bleu)

## Salva resultados da tarefa 

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.

In [ ]:
from joblib import dump

artifacts = {
    "model": marianmt_model
}

dump(artifacts, "/tmp/data/translate.joblib")